# Seminar Project: Speech and Article Prompting

---------------
``` 

Jakob Amann
Elias Gabriel Heppner, 1244414
Andri Rutschmann

***

In [3]:
# Setup
import numpy as np 
import pandas as pd 
import time
import pickle
import time
from tqdm.auto import tqdm
import nltk
# nltk.download('punkt')
from nltk.tokenize import word_tokenize # import tokenizer to estimate token count of a speech
import openai
import zipfile
import json

# api_keys = ["wAlKn00lAQIdM4R9nskS", "4hohJnlUY52Tqk24HXfh", '6uS6Kv0efLZLGsy0In3m'] # elias, jakob, andri
api_key = 'XbnxolhCiP1Stk0yWsJd'
openai.api_base = "http://91.107.239.71:80"
openai.api_key = api_key # choose api key

speeches = pd.read_csv('../data/opendiscourse/sub_speeches.zip', compression='zip')

C:\Users\Andri\envs\css\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Hyperparameters

In [14]:
max_tokens = 3000

## Politician Selection

In [4]:
# Define topics
topics_de = ["Waffenlieferungen an die Ukraine", "Covid-19 Restriktionen", 'Migration']
topics_en = ['Weapon exports to Ukraine', 'Covid-19 Restrictions', 'Migration']

In [5]:
# Filter the speeches on these topics
# speeches

In [6]:
# Select most active politicians
politicians = [{'name' : 'Alice Weidel', 'party' : 'AfD'},
              # {'name' : 'Alexander Gauland', 'party' : 'AfD'},
              {'name' : 'Olaf Scholz', 'party' : 'SPD'},
              # {'name' : 'Helge Lindh', 'party' : 'SPD'},
              {'name' : 'Jens Spahn', 'party' : 'CDU/CSU'},
              # {'name' : 'Angela Merkel', 'party' : 'CDU/CSU'},
              # {'name' : 'Konstantin Kuhle', 'party' : 'FDP'},
              {'name' : 'Christian Lindner', 'party' : 'FDP'},
              # {'name' : 'Volker Beck', 'party' : 'Grüne'},
              {'name' : 'Luise Amtsberg', 'party' : 'Grüne'},]

politician_df = pd.DataFrame(politicians)

newspapers = ['taz', 'Zeit', 'Süddeutsche', 'Spiegel', 'Welt'] # BILD?!

In [7]:
# Remove all speeches from other politicians
speeches = speeches[speeches['last_name'].isin([i['name'].split()[1] for i in politicians])]

### Prompt Setup

In [8]:
politician_df

,name,party
0,Alice Weidel,AfD
1,Olaf Scholz,SPD
2,Jens Spahn,CDU/CSU
3,Christian Lindner,FDP
4,Luise Amtsberg,Grüne


### Prompt Generation

## Impose speech length in some prompts and leave it out in others (impose a max)

In [11]:
# Function that generates prompt

def make_prompt(politician, topic, party=True, language='en', printing=False):
    if language == 'en' and topic in topics_de:
        raise ValueError('Please make sure the topic language and prompt language match.')

    if party:
        prompt_de = f'''Schreibe eine Rede für den Bundestag im Stil von {politician['name']} ({politician['party']}) zum Thema: {topic}.'''
        prompt_en = f'''Compose a speech for parliament in the style of {politician['name']} ({politician['party']}) on the topic: {topic}.'''

    if not party:
        prompt_de = f'''Schreibe eine Rede für den Bundestag im Stil von {politician['name']} zum Thema: {topic}.'''
        prompt_en = f'''Compose a speech for parliament in the style of {politician['name']} on the topic: {topic}.'''

    if language == 'en':
        out = prompt_en
    else:
        out = prompt_de

    for i in (('\n', ''), ('\\', '')): # replace new line chars and other nonsense...
        out = out.replace(*i)
        
    if printing:
        print(out)
        
    return out

In [12]:
make_prompt(politicians[0], topics_de[1], language='de')

'Schreibe eine Rede für den Bundestag im Stil von Alice Weidel (AfD) zum Thema: Covid-19 Restriktionen.'

In [13]:
# Function that prompts LLM
def llm_prompt(data, topic=None, party=True, language='en', n_runs=1, model='gpt-3.5-turbo-0125'):

    prompt = make_prompt(politician=data, topic=topic, party=party, language=language)

    while True:
        try:
            responses = openai.ChatCompletion.create(model=model,
                                                     messages=[{"role": "user", "content": prompt}],
                                                     max_tokens = int(avg_tokens * 1.5), # individual speech can be as much as 50% longer than the average
                                                     n=n_runs)
            
            # overall average length must be between 80% and 120% of original average
            # while (responses['usage']['completion_tokens'] / n_runs) < avg_tokens * 0.8 or (responses['usage']['completion_tokens'] / n_runs) > avg_tokens * 1.2:
            #     responses = openai.ChatCompletion.create(model=model,
            #                                          messages=[{"role": "user", "content": prompt}],
            #                                          max_tokens = int(avg_tokens * 1.5), # individual speech can be as much as 50% longer than the average
            #                                          n=n_runs)
            break

        except Exception as e:
                print(f'Error: {e}')
                continue

    print(avg_tokens)
    print(responses['usage']['completion_tokens'] / n_runs)
    if n_runs == 1:
        output = responses['choices'][0]['message']['content']
        for i in (('\n', ''), ('\\', '')): # replace new line chars and other nonsense
            output = output.replace(*i) # .lower()

    else:
        output = []
        for i in range(n_runs):
            run_response = responses['choices'][i]['message']['content']
            for i in (('\n', ''), ('\\', '')): # replace new line chars and other nonsense
                run_response = run_response.replace(*i) # .lower()
            output.append(run_response)
            
    return output

### Test Runs

In [29]:
llm_prompt(politicians[0], topics_de[0], language='de', n_runs=1)

850
572.0


'Meine Damen und Herren,ich stehe hier vor Ihnen, um über ein hochbrisantes Thema zu sprechen: die Waffenlieferungen an die Ukraine. Unser Land, Deutschland, ist in dieser Frage gespalten. Auf der einen Seite stehen die Befürworter dieser Lieferungen, die glauben, dass die Ukraine dringend unsere Unterstützung benötigt, um sich gegen die russische Aggression zu verteidigen. Auf der anderen Seite gibt es die Kritiker, die befürchten, dass Waffenlieferungen den Konflikt nur weiter anheizen und zu einer Eskalation führen könnten.Als Mitglied der Alternative für Deutschland setze ich mich dafür ein, dass Deutschland keine Waffen an die Ukraine liefert. Doch warum eigentlich nicht? Ist es nicht unsere Pflicht, einem souveränen Staat zu helfen, der von einem aggressiven Nachbarn bedroht wird?Ja, es ist wichtig, einem souveränen Staat zu helfen. Doch wir müssen auch die Folgen unseres Handelns bedenken. Waffenlieferungen könnten zu einer Eskalation des Konflikts führen und die Situation weite

## Get some data samples for testing

In [70]:
llm_speeches_sample = []

for i in politicians:
    out = {}
    out['name'] = i['name']
    out['party'] = i['party']
    out['llm_output'] = {}
    
    for j in topics_de:    
        out['llm_output'][j] = llm_prompt(i, j, language='de', n_runs=2)
    
    llm_speeches_sample.append(out)

llm_speeches_sample

850
714.0
850
699.0
850
730.0
850
720.0
850
642.0
850
639.0
850
686.0
850
653.0
850
725.0
850
644.5
850
669.0
850
645.0
850
689.5
850
646.5
850
733.0


[{'name': 'Alice Weidel',
  'party': 'AfD',
  'llm_output': {'Waffenlieferungen an die Ukraine': ['Sehr geehrte Damen und Herren,ich stehe heute vor Ihnen, um über die brisante Frage der Waffenlieferungen an die Ukraine zu sprechen. Diese Entscheidung hat weitreichende geopolitische Folgen und verdient eine sorgfältige Betrachtung. Es ist wichtig, dass wir uns bewusst sind, welche Konsequenzen unsere Handlungen haben können und wie wir am besten dazu beitragen können, den Frieden in der Region zu erhalten.Die Ukraine ist seit Jahren in einem blutigen Konflikt mit prorussischen Separatisten im Osten des Landes verwickelt. Die Kämpfe haben unzählige Menschenleben gefordert und das Land in eine schwierige Lage gebracht. Es ist verständlich, dass die ukrainische Regierung nach Unterstützung sucht, um ihre Souveränität und territoriale Integrität zu verteidigen. Doch müssen wir uns die Frage stellen, ob Waffenlieferungen tatsächlich der richtige Weg sind, um die Situation zu lösen.Waffenlie

In [71]:
with open('../data/llm_speeches_sample', 'w') as f:
    json.dump(llm_speeches_sample, f)